In [1]:

import pickle
import tensorflow as tf
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import cv2


In [2]:
# Function to load the model, embeddings, and filenames
def load_model_and_data():
    # Load the embeddings and filenames
    feature_list = np.array(pickle.load(open('embeddings.pkl', 'rb')))
    filenames = pickle.load(open('filenames.pkl', 'rb'))

    # Load the pre-trained ResNet50 model
    model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model.trainable = False  # Disable training for this model

    # Add a GlobalMaxPooling2D layer to the model
    model = tf.keras.Sequential([model, GlobalMaxPooling2D()])

    return model, feature_list, filenames


In [3]:
# Function to preprocess the input image
def preprocess_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    preprocessed_img = preprocess_input(expanded_img_array)  # Normalize the image for ResNet50
    return preprocessed_img


In [4]:
# Function to extract and normalize features from the image
def extract_features(model, preprocessed_img):
    # Extract the features using the model
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)  # Normalize the feature vector
    return normalized_result


In [5]:
# Function to find the nearest neighbors
def find_nearest_neighbors(feature_list, query_features, n_neighbors=6):
    # Use NearestNeighbors to find the most similar images
    neighbors = NearestNeighbors(n_neighbors=n_neighbors, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)  # Fit the Nearest Neighbors model
    distances, indices = neighbors.kneighbors([query_features])  # Get distances and indices of nearest neighbors
    return distances, indices


In [6]:
# Function to display images using matplotlib
def display_images(indices, filenames, start=1, end=6):
    # Loop through the nearest neighbors and display the images
    for file in indices[0][start:end]:  # Skip the first image (query image)
        temp_img = cv2.imread(filenames[file])  # Read image using OpenCV
        temp_img_rgb = cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for matplotlib
        plt.imshow(cv2.resize(temp_img_rgb, (512, 512)))  # Resize image for display
        plt.axis('off')  # Hide axes
        plt.show()  # Show the image


In [8]:
# Main script execution
if __name__ == "__main__":
    try:
        # Load model and pre-saved data
        model, feature_list, filenames = load_model_and_data()

        # Preprocess the image to be queried
        preprocessed_img = preprocess_image('1164.jpg')

        # Extract the features from the preprocessed image
        query_features = extract_features(model, preprocessed_img)

        # Ensure the extracted features are a 1D array of 2048 elements
        if query_features.shape[0] != 2048:
            raise ValueError(f"Expected 2048 features, got {query_features.shape[0]}")

        # Find the nearest neighbors based on the extracted features
        distances, indices = find_nearest_neighbors(feature_list, query_features)

        # Print the indices of the nearest images (for debugging)
        print("Indices of nearest images:", indices)

        # Display the nearest images using matplotlib
        display_images(indices, filenames)

    except Exception as e:
        print("An error occurred:", str(e))


An error occurred: [Errno 2] No such file or directory: 'embeddings.pkl'
